Libs

In [15]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

Global variables

In [16]:
chrome_options = ChromeOptions()
chrome_options.add_argument("--headless=new")
main_df = pd.DataFrame(columns = ["Stadium", "Capacity", "Spectators", "Average",	"Matches", "sold out", "Capacity", "Club", "Year"])
seasons = [2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
leagues_prefixs = ["SA1", "SA2L"]
dfs = []

Main function

In [19]:
def extract_attendences():
  for i in range(len(seasons)):
    url = f'https://www.transfermarkt.com/saudi-pro-league/besucherzahlen/wettbewerb/SA1/saison_id/{seasons[i]}/plus/1'
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    html = scrapingHTML(driver, i)
    print(f'Season {seasons[i]} - Scraping done!')

NoSuchElementExceptionHandler function

In [25]:
def NoSuchElementExceptionHandler(driver, ID, XPATH):
  try:
        WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it(driver.find_element(By.ID, ID)))
        driver.find_element(By.XPATH, XPATH).click()
  except NoSuchElementException as exc:
    pass 

Function to get data with Selenium and BS4

In [18]:
def scrapingHTML(driver, i):
  NoSuchElementExceptionHandler(driver, "sp_message_iframe_851946", "//*[@id='notice']/div[3]/div[3]/button")

  table = driver.find_element(By.XPATH, '//*[@id="yw1"]/table')
  data = table.get_attribute('outerHTML')
  soup = BeautifulSoup(data, 'html.parser')
  html = soup.find(name='table')
  df = pd.read_html(str(html))[0]
  df["Club"] = ""
  df["Year"] = seasons[i]
  dfs.append(df)
  driver.quit()
  return html

Extract Teams info

In [28]:
def teams_list_extract(league):
  for i in range(len(league)):
    url = f"https://www.transfermarkt.us/saudi-pro-league/startseite/wettbewerb/{league[i]}"
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)

    NoSuchElementExceptionHandler(driver, "sp_message_iframe_851946", "//*[@id='notice']/div[3]/div[3]/button")

    table = driver.find_element(By.XPATH, '//*[@id="yw1"]/table')
    data = table.get_attribute('outerHTML')
    soup = BeautifulSoup(data, 'html.parser')
    html = soup.find(name='table')
    df = pd.read_html(str(html))[0]
    df["League"] = league[i]
    df.to_csv(f"./RAW/ArabicSoccer/Teams/{league[i]}_teams_2023.csv", encoding='utf-8', index=False)

Extract All Champions - SPL

In [26]:
def extract_champions(league):
  for i in range(len(league)):
    url = f"https://www.transfermarkt.us/saudi-professional-league/erfolge/wettbewerb/{league[i]}"
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)

    NoSuchElementExceptionHandler(driver, "sp_message_iframe_851946", "//*[@id='notice']/div[3]/div[3]/button")
    
    try:
        WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it(driver.find_element(By.ID, "sp_message_iframe_851946")))
        driver.find_element(By.XPATH,"").click()
    except NoSuchElementException as exc:
      pass

    table = driver.find_element(By.XPATH, '//*[@id="yw1"]/table')
    data = table.get_attribute('outerHTML')
    soup = BeautifulSoup(data, 'html.parser')
    html = soup.find(name='table')
    df = pd.read_html(str(html))[0]
    if i == 1:
      df["League"] = "Saudi Pro League"
    else:
      df["League"] = "Yelo League"
    df.to_csv(f"./RAW/ArabicSoccer/Champions/{league[i]}_Champions.csv", encoding='utf-8', index=False)

Exec

In [ ]:
extract_champions(leagues_prefixs)
extract_attendences()
teams_list_extract(leagues_prefixs)

Concat DataFrame list and create CSV

In [ ]:
df_og = pd.concat(dfs)
df_og.to_csv("./RAW/ArabicSoccer/Attendances/Arabic_Attendances_2007_2023.csv", encoding='utf-8', index=False)